In [ ]:
from tweepy import Stream 
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

In [ ]:
CONSUMER_KEY = "mDU9oAfCJWSHrWroaYxnUlzxK"
CONSUMER_SECRET = "AExpaDFprBZhfqPBNC1v1f5UH0ECDTXXUUzUf9cj6G3laKjdI7"
ACCESS_TOKEN = "1052201566776815622-MKTrQfe21jl60m6oYu72glLe8Ei66L"
ACCESS_TOKEN_SECRET = "jrIsjMYaFjfnHZwoITlvarCqiTSh6Fne48tTeu8nuJu3l"

In [ ]:
import socket
import sys 
import requests
import requests_oauthlib
import json

In [ ]:
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY,CONSUMER_SECRET,ACCESS_TOKEN,ACCESS_TOKEN_SECRET)

In [ ]:
def get_tweets():
    url='https://stream.twitter.com/1.1/statuses/filter.json'
    query_data=[('language','en'),('track','%23LouWilliams'),('track','%23StephenCurry'),('track','%23KevinDurant'),('track','%23Cousins'), \
               ('track','%23NBA'),('track','NBA'),('track','Lou Williams'),('track','Stephen Curry'),('track','Kevin Durant'),('track','Cousins')]
    query_url=url+'?'+'&'.join([str(t[0])+'='+str(t[1]) for t in query_data])
    response=requests.get(query_url,auth=my_auth,stream=True)
    print(query_url,response)
    return response

In [ ]:
def send_tweets_to_spark(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text']
            print(tweet_text)
            tcp_connection.send((tweet_text + '\n').encode())
        except:
            e = sys.exc_info()[0]
            print(e)

In [ ]:
TCP_IP = "localhost"
TCP_PORT = 9009
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
resp = get_tweets()
send_tweets_to_spark(resp, conn)